# Introduction

For the past week you have been learning about new Python libraries: `Numpy` and `Pandas`. Up until now we haven't talked too much about what makes these libraries useful. Yeah, `Numpy` is good for vectorizing computations, and `Pandas` is good for loading and manipulating data; but are those tasks ends, in and of themselves, or means to an end? 

The answer, as with many of our answers is, it depends. So far you've seen both libraries used independent of any particular task. Tonight that changes. Tonight we will use these libraries in efforts to accomplish the same end as [Pinky and the Brain](https://en.wikipedia.org/wiki/Pinky_and_the_Brain)! Ok, maybe be won't be trying to take over the world (... or will we?), but we will be working on just as lofty an endeavour: predicting the unknown.

## Types of Learning

In the realm of Data Science we talk about two distinct types of learning from data, [supervised](https://en.wikipedia.org/wiki/Supervised_learning) and [unsupervised](https://en.wikipedia.org/wiki/Unsupervised_learning). There are other sub-classes of these two types of learning, but for the most part any data science pursuit can be classified into one of these two. **Unsupervised learning** consists of trying to find unapparent structure in your data that holds meaningful information. Today we are going to be focusing on **supervised learning**, though. This brand of learning attempts to build a model that predicts a target outcome from known inputs. This is accomplished through the use of inputs with correct targets to train the model. For example, if I'm trying to predict whether or not there will be a traffic jam on the highway. Data to solve this problem might have inputs such as the day of the week, the time of day, whether or not it's a holiday, if there is a major event going on that day, etc; and a target of traffic jam or not. Once the model is trained, we'll get to how to do this soon, I can then predict if there's going to be a traffic jam or not by plugging in the the desired inputs to the model. 

This was an example of what is known as a **binary classification** problem, the binary comes from deciding yes or no on traffic jam. Binary is just the simplest case of **classification problems**, though. Classification problems can tackle problems where you are trying to predict if inputs cause something to be in any of `n` classes. We will look at an example of this later with the iris data set.

We don't always just want to classify something as a target for our predictions, though. Frequently we want predict how much of something will happen based on inputs. This type of problem is know as **regression**. An example of this is trying to predict the price of a stock at the end of a day based on how the market behaved that day.